In [ ]:
!pip install pygame

In [ ]:
import gradio as gr
import os
from dotenv import load_dotenv
from openai import OpenAI
import time

client = OpenAI()
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
import speech_recognition as sr

r = sr.Recognizer()

with sr.Microphone() as source:
    print('Speak Anything :')
    audio = r.listen(source)
    
    try:
        text = r.recognize_google(audio, language='ko-KR')
        print('You said : {}'.format(text))
    except sr.UnknownValueError:
        print('Sorry could not recognize your voice')
    except sr.RequestError as e:
        print('Could not request results from Google Speech Recognition service; {0}'.format(e))


In [ ]:
import speech_recognition as sr
import gradio as gr
import os
from dotenv import load_dotenv
from openai import OpenAI
from gtts import gTTS
import pygame
import time

client = OpenAI()
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

#OpenAI API에게서 응답을 받기 위한 코드
def get_response_from_openai(text):
    messages = [{"role": "user", "content": text}]
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
    )
    return response.choices[0].message.content


# 음성 인식을 위한 설정
r = sr.Recognizer()

with sr.Microphone() as source:
    print('Speak Anything :')
    audio = r.listen(source)
    
    try:
        text = r.recognize_google(audio, language='ko-KR')
        print('You said : {}'.format(text))
        
        # OpenAI API로 응답 받기
        response = get_response_from_openai(text)
        print('OpenAI response: ', response)

        tts = gTTS(text=response, lang='ko')
        tts.save("response.mp3")
        
        # Pygame을 사용하여 음성 파일 재생
        pygame.mixer.init()
        pygame.mixer.music.load("response.mp3")
        pygame.mixer.music.play()

        # 음악 재생이 완료될 때까지 대기
        while pygame.mixer.music.get_busy():
            pygame.time.Clock().tick(10)
        
    except sr.UnknownValueError:
        print('Sorry could not recognize your voice')
    except sr.RequestError as e:
        print('Could not request results from Google Speech Recognition service; {0}'.format(e))

In [ ]:
import gradio as gr
import speech_recognition as sr
from gtts import gTTS
import os
from dotenv import load_dotenv
from openai import OpenAI
import tempfile

# 환경 변수 로드
load_dotenv()
client = OpenAI()
api_key = os.getenv("OPENAI_API_KEY")


# OpenAI API로부터 응답 받는 함수
def get_response_from_openai(text):
    messages = [{"role": "user", "content": text}]
    
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
    )
    return response.choices[0].message.content

# 음성 인식 및 음성 응답 생성
def process_voice(file):
    r = sr.Recognizer()
    with sr.AudioFile(file) as source:
        audio = r.record(source)
    
    try:
        # 음성 인식
        text = r.recognize_google(audio, language='ko-KR')
        # OpenAI API로 응답 받기
        response = get_response_from_openai(text)
        # 응답을 음성으로 변환
        tts = gTTS(text=response, lang='ko')
        
        # 음성을 임시 파일로 저장
        temp_file = tempfile.NamedTemporaryFile(delete=False, suffix='.mp3')
        tts.save(temp_file.name)
        temp_file.close()  # 파일 사용이 끝난 후 파일을 닫아야 함
        return temp_file.name  # 파일 경로 반환
    except sr.UnknownValueError:
        return "Sorry, could not recognize your voice."
    except sr.RequestError as e:
        return f"Could not request results from Google Speech Recognition service; {e}"

# 그레디오 인터페이스 설정
demo = gr.Interface(
    fn=process_voice,
    inputs=gr.Audio(type="filepath", label="Record your question"),
    outputs=gr.Audio(label="Response"),
    title="Voice to OpenAI Response",
    description="Speak and receive a response from OpenAI."
)

# 인터페이스 실행
demo.launch()


In [ ]:
import gradio as gr
import speech_recognition as sr
from gtts import gTTS
import os
from dotenv import load_dotenv
from openai import OpenAI
import tempfile

# 환경 변수 로드
load_dotenv()
client = OpenAI()
api_key = os.getenv("OPENAI_API_KEY")

history = []
# OpenAI API로부터 응답 받는 함수
def get_response_from_openai(text):
    global history
    history.append({"role": "user", "content": text})
    
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=history,
    )
    history.append({"role":"assistant","content":response.choices[0].message.content})
    return response.choices[0].message.content, history

# 음성 인식 및 음성 응답 생성
def process_voice(file, reset_history=False):
    if reset_history:
        global history
        history=[]

    r = sr.Recognizer()
    with sr.AudioFile(file) as source:
        audio = r.record(source)
    
    try:
        # 음성 인식
        text = r.recognize_google(audio, language='ko-KR')
        # OpenAI API로 응답 받기
        response, history_display = get_response_from_openai(text)
        # 응답을 음성으로 변환
        tts = gTTS(text=response, lang='ko')
        
        # 음성을 임시 파일로 저장
        temp_file = tempfile.NamedTemporaryFile(delete=False, suffix='.mp3')
        tts.save(temp_file.name)
        temp_file.close()  # 파일 사용이 끝난 후 파일을 닫아야 함
        return temp_file.name, history_display  # 파일 경로 반환
    except sr.UnknownValueError:
        return "목소리를 인식하지 못했습니다.", history
    except sr.RequestError as e:
        return f"Speech Recognition으로부터 결과를 받을 수 없습니다. {e}", history

# Gradio 인터페이스 설정
demo = gr.Interface(
    fn=process_voice,
    inputs=[gr.Audio(type="filepath", label="질문"),gr.Checkbox(label="대화 내역 지우기")],
    outputs=[gr.Audio(label="AI비서"),gr.JSON(label="대화 내역")],
    title="음성 AI비서",
    description="음성으로 AI와 대화를 나누어보세요"
)

# 인터페이스 실행
demo.launch()


Running on local URL:  http://127.0.0.1:7902

To create a public link, set `share=True` in `launch()`.


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "c:\anaconda\Lib\site-packages\uvicorn\protocols\http\h11_impl.py", line 407, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\anaconda\Lib\site-packages\uvicorn\middleware\proxy_headers.py", line 69, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\anaconda\Lib\site-packages\fastapi\applications.py", line 1054, in __call__
    await super().__call__(scope, receive, send)
  File "c:\anaconda\Lib\site-packages\starlette\applications.py", line 123, in __call__
    await self.middleware_stack(scope, receive, send)
  File "c:\anaconda\Lib\site-packages\starlette\middleware\errors.py", line 186, in __call__
    raise exc
  File "c:\anaconda\Lib\site-packages\starlette\middleware\errors.py", line 164, in __call__
    await self.app(scope, receive, _s